导入包和库

In [16]:
import os
import sys
from typing import List

import numpy as np
import pandas as pd

sys.path.append(r"D:\\QUANT_GAME\\python_game\\pythonProject")
from my_tools_packages import MyDecorator as MD

running_time = MD.running_time

sys.path.append(r"D:\\QUANT_GAME\\python_game\\pythonProject\\hk_stk_data_processing_codes_by_jsy\\common")
import config
import utilities as utl
from EvaluateDateBar import EvaluateDateBar as EvalDateBar



配置文件

In [17]:
hk_date_bar_version = 'v11'
hk_date_bar_file_path = config.get_config('date_bar_develop_hist_file_path')[hk_date_bar_version]
hk_date_bar = pd.read_pickle(hk_date_bar_file_path)
df = hk_date_bar.copy()
print('总的有4889273根日线数据')

总的有4889273根日线数据


在日线上计算amount/volume,和其余价格的相对误差比对

In [18]:
df['amount/volume'] = df['amount']/df['volume']


amount/volume为nan、inf、0，有以下数据

In [19]:
# amount/volume为nan
amount_volume_is_na = df[df['amount/volume'].isna()]
print('有367根日线数据，但不一定为异常')
print('amount==volume==0')

有367根日线数据，但不一定为异常
amount==volume==0


In [20]:
# amount/volume为inf
amount_volume_is_inf = df[df['amount/volume']>999999999]
print('有339根日线数据，且为异常')
print('amount!=0,volume==0')

有339根日线数据，且为异常
amount!=0,volume==0


In [21]:
# amount/volume为0
amount_volume_is_0 = df[df['amount/volume']==0]
print('有103根日线数据，且为异常')
print('amount==0,volume!=0')

有103根日线数据，且为异常
amount==0,volume!=0


由于以上错误只是在量上，无法判断量价关系，而现在要通过分析量价关系的相对误差来找出错误的原因，以上这几类可以暂时忽略，共剔除80

In [22]:
df_t = df[~df.index.isin((amount_volume_is_na.index)|(amount_volume_is_0.index)|(amount_volume_is_inf.index))]
price_and_amount_volume_real_err = (df_t[['open','high','low','close']]/df_t[['amount/volume']].values-1).abs()
df_t_num = len(df_t)
print(f'剔除暂时不考虑的错误，剔除后有{df_t_num}')


C:\Users\win\AppData\Local\Temp\ipykernel_13796\251065794.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  df_t = df[~df.index.isin((amount_volume_is_na.index)|(amount_volume_is_0.index)|(amount_volume_is_inf.index))]
C:\Users\win\AppData\Local\Temp\ipykernel_13796\251065794.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  df_t = df[~df.index.isin((amount_volume_is_na.index)|(amount_volume_is_0.index)|(amount_volume_is_inf.index))]


剔除暂时不考虑的错误，剔除后有4888464


In [23]:
# 相对差为0
real_err_is_0 = price_and_amount_volume_real_err[(price_and_amount_volume_real_err==0).all(axis=1)]
real_err_df_is_0 = df_t[(price_and_amount_volume_real_err==0).all(axis=1)]
real_err_is_0_num = len(real_err_is_0)
print(f'准确无误的数据,有{real_err_is_0_num}根日线数据')
print('都在20150610之前')

准确无误的数据,有195872根日线数据
都在20150610之前


In [28]:
check_df = df_t[~df_t.index.isin(real_err_df_is_0.index)]
check_df_real_err = price_and_amount_volume_real_err.loc[check_df.index,:]
# 剔除相对误差为0的准确无误的数据后有
check_df_num = len(check_df)
print(f'剔除准确的数据后，剩下的数据有{check_df_num}')

剔除相对误差后，准确无误的数据有4692592


In [33]:
# 非常准确的数据，相对误差0.01以内的有
real_err_is_01 = price_and_amount_volume_real_err[(price_and_amount_volume_real_err<0.01).all(axis=1)]
real_err_df_is_01 = df_t[(price_and_amount_volume_real_err<0.01).all(axis=1)]
real_err_is_01_num = len(real_err_is_01)
print(f'非常精准<0.01,的数据有{real_err_is_01_num}')


非常精准<0.01,的数据有591217


In [41]:
# 较准确的数据，相对误差0.05以内的有
real_err_is_05 = price_and_amount_volume_real_err[(price_and_amount_volume_real_err<=0.05).all(axis=1)]
real_err_df_is_05 = df_t[(price_and_amount_volume_real_err<=0.05).all(axis=1)]
real_err_is_05_num = len(real_err_is_05)
print(f'较精确的数据<=0.05的数据有{real_err_is_05_num}')

较精确的数据<0.05的数据有2080050


In [42]:
# 不考虑较准确的数据，剩下需要处理的数据有
check_df_1 = check_df[~check_df.index.isin(real_err_df_is_05.index)]
check_df_1_real_err = price_and_amount_volume_real_err.loc[check_df_1.index,:]

check_df_1_num = len(check_df_1)
print(f'不考虑较准确的数据，剩下需要处理的数据有{check_df_1_num}')

不考虑较准确的数据，剩下需要处理的数据有2808414


再剔除volume100处理后小于0.05的有

In [43]:
real_err_is_95_105 = price_and_amount_volume_real_err[(price_and_amount_volume_real_err>=0.95).all(axis=1)
                                                  &(price_and_amount_volume_real_err<=1.05).all(axis=1)]
real_err_df_is_95_105 = df_t[(price_and_amount_volume_real_err>=0.95).all(axis=1)
                                                  &(price_and_amount_volume_real_err<=1.05).all(axis=1)]
real_err_is_95_105_num = len(real_err_is_95_105)
print(f'可通过volume10处理的数据有{real_err_is_95_105_num}')


In [45]:
check_df_2 = check_df_1[~check_df_1.index.isin(real_err_df_is_95_105.index)]
check_df2_real_err = price_and_amount_volume_real_err.loc[check_df_2.index,:]

check_df_2_num = len(check_df_2)
print(f'不考虑可以通过volume100处理的数据有{check_df_2_num}')

不考虑可以通过volume100处理的数据有378748


把阈值提升到0.1

In [47]:
real_err_is_1 = price_and_amount_volume_real_err[(price_and_amount_volume_real_err<=0.1).all(axis=1)]
real_err_df_is_1 = df_t[(price_and_amount_volume_real_err<=0.1).all(axis=1)]

real_err_is_1_num = len(real_err_is_1)


In [48]:
real_err_is_90_110 = price_and_amount_volume_real_err[(price_and_amount_volume_real_err>=0.9).all(axis=1)
                                                  &(price_and_amount_volume_real_err<=1.1).all(axis=1)]
real_err_df_is_90_110 = df_t[(price_and_amount_volume_real_err>=0.9).all(axis=1)
                                                  &(price_and_amount_volume_real_err<=1.1).all(axis=1)]
real_err_is_90_110_num = len(real_err_is_95_105)

In [50]:
check_df_3 = check_df[(~check_df.index.isin(real_err_df_is_1.index))&(~check_df.index.isin(real_err_df_is_90_110.index))]
check_df3_real_err = price_and_amount_volume_real_err.loc[check_df_3.index,:]